# Hiragana convolutional network

The Hiragana datatset is composed of the 46 japanese characters called hiragana. Each folder is named after the hiragana name and contains 100 jpg images of handwritten hiragana.

In [31]:
import os
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from collections import defaultdict

In [32]:
import torch

In [33]:
import random

In [34]:
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
import numpy as np
from tqdm import tqdm

## Set seed

In [35]:
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

## Dataset

In [36]:
DATA_DIR = "hiragana-dataset"
BATCH_SIZE = 32
IMAGE_SIZE = 64
NUM_CLASSES = 46
EPOCHS = 15
LR = 1e-3

In [37]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

Using device: cpu


## Data augmentation

Data augmentation consist in small processing steps like small rotation, scaling etc. Be careful to not do any horizontal flip as it might change the meaning of the hiragana. the hiragana images are already inn black and white.

In [38]:
train_transform =  transforms.Compose([
    transforms.Grayscale(num_output_channels=1),
    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
    transforms.RandomRotation(10),
    transforms.RandomAffine(
        degrees=0,
        translate=(0.1, 0.1),
        scale=(0.9, 1.1)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

In [39]:
val_transform = transforms.Compose([transforms.Grayscale(num_output_channels=1),
                                    transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
                                    transforms.ToTensor(),
                                    transforms.Normalize((0.5,), (0.5,))])

## Load the hiragana dataset with Imagefolder

In [40]:
full_dataset = datasets.ImageFolder(root=DATA_DIR, transform=train_transform)
print("Classes:", full_dataset.classes)
print("Number of images:", len(full_dataset))

Classes: ['aa', 'chi', 'ee', 'fu', 'ha', 'he', 'hi', 'ho', 'ii', 'ka', 'ke', 'ki', 'ko', 'ku', 'ma', 'me', 'mi', 'mo', 'mu', 'na', 'ne', 'ni', 'nn', 'no', 'nu', 'oo', 'ra', 're', 'ri', 'ro', 'ru', 'sa', 'se', 'shi', 'so', 'su', 'ta', 'te', 'to', 'tsu', 'uu', 'wa', 'wo', 'ya', 'yo', 'yu']
Number of images: 4600


## Stratified Train/Validation split 

I want to make sure I have all the classes represented in each set.

In [41]:
def stratified_split(dataset, val_ratio = 0.2, seed=42):
    np.random.seed(seed)
    class_indices = defaultdict(list)
    
    for idx, (_,label) in enumerate(dataset.samples):
        class_indices[label].append(idx)
        
    train_idx, val_idx = [], []
    for label, indices in class_indices.items():
        np.random.shuffle(indices)
        split = int(len(indices) * (1 - val_ratio))
        train_idx.extend(indices[:split])
        val_idx.extend(indices[split:])
        
    return train_idx, val_idx

train_idx, val_idx = stratified_split(full_dataset, val_ratio=0.2)

train_dataset = Subset(full_dataset, train_idx)
val_dataset = Subset(datasets.ImageFolder(DATA_DIR, transform=val_transform),
                     val_idx)

In [42]:
print("Train size:", len(train_dataset))
print("Validation size:", len(val_dataset))

Train size: 3680
Validation size: 920


## DataLoaders

In [43]:
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)

In [44]:
images, labels = next(iter(train_loader))
print(images.shape) # e.g., torch.Size([32, 1, 64, 64])
print(labels[:5])

torch.Size([32, 1, 64, 64])
tensor([35,  9, 42, 27, 19])


## Hiragana CNN model

### Model definition

In [45]:
class HiraganaCNN(nn.Module):
    def __init__(self, num_classes=46):
        super().__init__()
        
        self.features = nn.Sequential(
            nn.Conv2d(1, 32, 3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2),
            
            nn.Conv2d(32, 64, 3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2),
            
            nn.Conv2d(64, 128, 3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2)
        )
        
        self.classifier = nn.Sequential(
            nn.Linear(128 * 8 * 8, 512),
            nn.ReLU(),
            nn.Dropout(0.5),
            nn.Linear(512, num_classes)
        )
        
    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

## Initialize the model

In [46]:
model = HiraganaCNN(NUM_CLASSES).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LR) # LR = 1e-3
print(model)

HiraganaCNN(
  (features): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Linear(in_features=8192, out_features=512, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(

## Training

In [47]:
for epoch in range(EPOCHS):
    model.train()
    train_loss=0
    
    for images, labels in tqdm(train_loader):
        images, labels = images.to(device), labels.to(device)
        
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        
    # Validation part
    model.eval()
    correct = 0
    total = 0
    
    with torch.no_grad():
        for images, labels in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (preds == labels).sum().item()
    
    acc = 100 * correct / total
    print(f"Epoch [{epoch+1}/{EPOCHS}]" f"Loss: {train_loss:.3f} | Val Acc: {acc:.2f}%")
    

100%|██████████| 115/115 [00:32<00:00,  3.58it/s]


Epoch [1/15]Loss: 452.175 | Val Acc: 18.37%


100%|██████████| 115/115 [00:29<00:00,  3.89it/s]


Epoch [2/15]Loss: 328.800 | Val Acc: 76.52%


100%|██████████| 115/115 [00:30<00:00,  3.83it/s]


...

Epoch [13/15]Loss: 146.109 | Val Acc: 99.35%


100%|██████████| 115/115 [00:30<00:00,  3.81it/s]


Epoch [14/15]Loss: 147.287 | Val Acc: 99.78%


100%|██████████| 115/115 [00:29<00:00,  3.86it/s]


Epoch [15/15]Loss: 140.494 | Val Acc: 100.00%


## Prediction

### Class mapping

In [49]:
idx_to_class = {v: k for k, v in full_dataset.class_to_idx.items()}
print(idx_to_class)

{0: 'aa', 1: 'chi', 2: 'ee', 3: 'fu', 4: 'ha', 5: 'he', 6: 'hi', 7: 'ho', 8: 'ii', 9: 'ka', 10: 'ke', 11: 'ki', 12: 'ko', 13: 'ku', 14: 'ma', 15: 'me', 16: 'mi', 17: 'mo', 18: 'mu', 19: 'na', 20: 'ne', 21: 'ni', 22: 'nn', 23: 'no', 24: 'nu', 25: 'oo', 26: 'ra', 27: 're', 28: 'ri', 29: 'ro', 30: 'ru', 31: 'sa', 32: 'se', 33: 'shi', 34: 'so', 35: 'su', 36: 'ta', 37: 'te', 38: 'to', 39: 'tsu', 40: 'uu', 41: 'wa', 42: 'wo', 43: 'ya', 44: 'yo', 45: 'yu'}


### Predict one image

In [50]:
from PIL import Image

In [53]:
def predict_image(image_path):
    img = Image.open(image_path)
    img = val_transform(img).unsqueeze(0).to(device)
    
    model.eval()
    with torch.no_grad():
        output = model(img)
        pred = torch.argmax(output, 1).item()
        
    return idx_to_class[pred]

In [54]:
print(predict_image("hiragana-dataset/aa/drawing_20250805_081540.jpg"))

aa


## Save model

In [55]:
torch.save(model.state_dict(), "hiragana_cnn.pth")